In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import sklearn.metrics as skm
import wandb
import torch

from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from tsai.all import *
from fastai.callback.wandb import *
from visualization_functions import *
from fastai.layers import *
from sklearn.metrics import accuracy_score
from tqdm.notebook import trange, tqdm

from tsai.all import *
from fastai.callback.wandb import *
from visualization_functions import *

/home/iris/yg5d6/.conda/envs/tsai/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/iris/yg5d6/.conda/envs/tsai/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


ModuleNotFoundError: No module named 'visualization_functions'

In [2]:
computer_setup()

os             : Linux-4.15.0-191-generic-x86_64-with-debian-buster-sid
python         : 3.7.12
tsai           : 0.3.1
fastai         : 2.6.3
fastcore       : 1.4.3
torch          : 1.11.0+cu102
device         : 1 gpu (['NVIDIA GeForce RTX 2060'])
cpu cores      : 12
RAM            : 62.85 GB
GPU memory     : [5.79] GB


In [3]:
class FeatureMLP(nn.Module):
    
    def __init__(self):
        super(FeatureMLP, self).__init__()
        self.flatten = Reshape(-1)
        self.mlp = nn.ModuleList()
        self.mlp.append(LinBnDrop(48,50,bn=True, p=0.1, act=get_act_fn(nn.ReLU(inplace=True)), lin_first=False))
        self.mlp.append(LinBnDrop(50,50,bn=True, p=0.2, act=get_act_fn(nn.ReLU(inplace=True)), lin_first=False))
        self.mlp.append(LinBnDrop(50,50,bn=True, p=0.2, act=get_act_fn(nn.ReLU(inplace=True)), lin_first=False))
        self.head = nn.Sequential(LinBnDrop(50,2, bn=False, p=0))
    
    def forward(self, x):
        x = self.flatten(x)
        for mlp in self.mlp: x = mlp(x)
        return self.head(x)   

In [4]:
def LoadTrainTestFeatures(FEAT, LABEL, sub_test):
    num_signal = np.shape(FEAT[sub_test,0])[0]    
    X_Temp     = FEAT[sub_test,0]
    Y_Temp     = LABEL[sub_test,0].flatten()

    num_leftout = round(leftout*num_signal)
    index_leftout = np.random.choice(range(num_signal), size=num_leftout, replace=False)
    print("Left-out Test samples: ", index_leftout.size)

    X_Test = X_Temp[index_leftout,:]
    Y_Test = Y_Temp[index_leftout]

    index_include = np.arange(num_signal)
    index_include = np.delete(index_include, index_leftout)
    print("Included Training samples: ", index_include.size)
    
    X_include = X_Temp[index_include,:]
    Y_include = Y_Temp[index_include]

    # ===== Load Traing Signals =====
    X_Train = np.zeros((0,feature_dimension))
    Y_Train = np.zeros(0)    
    for sub_train in range(40):
        if sub_train != sub_test:
            x_s = FEAT_N[sub_train,0]
            y_s = LABEL[sub_train,0].flatten()
            # ===== CAN BE CONVERTED INTO A FUNCTION =====
            X_Train = np.concatenate((X_Train, x_s), axis=0)
            Y_Train = np.concatenate((Y_Train, y_s), axis=0)

    X_Train = np.concatenate((X_Train, X_include), axis=0)
    Y_Train = np.concatenate((Y_Train, Y_include), axis=0)        

    print('# of Healthy Samples: %d'%(np.sum(Y_Train == -1)))
    print('# of Fatigued Samples: %d'%(np.sum(Y_Train == 1)))   
    
    return X_Train, Y_Train, X_Test, Y_Test

## Load Data

In [5]:
# DATA_ALL = sio.loadmat("subjects_88_v2.mat")

DATA_ALL = sio.loadmat("subjects_40_v6.mat")
VFI_LIST = sio.loadmat("Notes/VFI_new_matched.mat")

VFI_1       = np.ravel(VFI_LIST['VFI1'])
VFI_2       = np.ravel(VFI_LIST['VFI2'])
SIG         = DATA_ALL['DATA']              # raw sEMG signals
FEAT        = DATA_ALL['FEAT']              # Orignally calculated features
FEAT_N      = DATA_ALL['FEAT_N']            # Normalized features
LABEL       = DATA_ALL['LABEL']             # Labels
SUBJECT_ID  = DATA_ALL['SUBJECT_ID']        # Sujbect ID
LABEL_VOWEL = DATA_ALL['LABEL_VOWEL']
VFI         = DATA_ALL['SUBJECT_VFI']

In [6]:
wand_config = 0
leftout = 1
# window_length = 4000
feature_dimension = 48
training_acc = np.zeros(40)
testing_acc  = np.zeros(40)

In [8]:
# X_All = np.zeros((0,4,window_length))
X_All = np.zeros((0,feature_dimension))

Y_All = np.zeros(0)    
for sub_train in range(40):
    
    x_s = FEAT_N[sub_train,0]
    y_s = LABEL[sub_train,0].flatten()

    X_All = np.concatenate((X_All, x_s), axis=0)
    Y_All = np.concatenate((Y_All, y_s), axis=0)

print('Total data and labels:')                
print(X_All.shape, Y_All.shape)
print('# of Healthy  Samples: %d'%(np.sum(Y_All == -1)))
print('# of Fatigued Samples: %d'%(np.sum(Y_All == 1)))    

Total data and labels:
(6472, 48) (6472,)
# of Healthy  Samples: 3202
# of Fatigued Samples: 3270


In [ ]:
sub_index = [0,1,2,3,4,20,21,22,23,24]
for sub_test in sub_index:
# for sub_test in range(40):
    sub_txt = "R%03d"%(int(SUBJECT_ID[sub_test][0][0]))
    print('Test Subject %s:'%(sub_txt))
    print('VFI-1:', (VFI[sub_test][0][0]))
    if int(VFI[sub_test][0][0]) > 10:
        sub_group = 'Fatigued'
    else:
        sub_group = 'Healthy'

    if wand_config == 1:
        run = wandb.init(project="sEMG Leave-One-Out Classification 40 100 Feature Base TSAI", 
                         group=sub_group,
                         reinit=True, 
                         name=sub_txt)

    #  Load training/testing features    
    X_Train, Y_Train, X_Test, Y_Test = LoadTrainTestFeatures(FEAT_N, LABEL, sub_test)

    #  Data initialization for fastai/pytorch
    splits = get_splits(Y_Train, valid_size=.1, stratify=True, random_state=23, shuffle=True, show_plot=False)
    tfms  = [None, [Categorize()]]
    dsets = TSDatasets(X_Train, Y_Train, tfms=tfms, splits=splits)
    
    dls = TSDataLoaders.from_dsets(dsets.train, 
                                   dsets.valid, 
                                   shuffle_train=True,
                                   bs=32,
                                   num_workers=0)    

    # dls.show_batch()

    #  Model definitions 
    model = FeatureMLP()
    
    if wand_config == 1:
        learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy, cbs=WandbCallback())
    else:
        learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)


    # Training
    with learn.no_logging():
#         lr = learn.lr_find(stop_div=False, num_it=200)
#         learn.fit_one_cycle(50, lr_max=0.001)
        learn.fit(50, lr=0.001)
    

    # learn.recorder.plot_metrics()

    
    
    
    inp, train_probas, _, train_preds = learn.get_X_preds(X_Train.reshape((-1,1,48)), 
                                                          with_input=True, 
                                                          with_decoded=True)
    preds = np.fromstring(train_preds[1:-1], sep=',')
    training_acc[sub_test] = skm.accuracy_score(Y_Train, preds)  
    
    
    
    
    print('Validation accuracy:%10.6f'%(learn.validate()[1]))
    
    # the prediction returned by the function for some reason was in string so needed to be converted.
    inp, test_probas , _, test_preds  = learn.get_X_preds(X_Test.reshape((-1,1,48)), 
                                                          with_input=True, 
                                                          with_decoded=True)
    preds = np.fromstring(test_preds[1:-1], sep=',')
    testing_acc[sub_test] = skm.accuracy_score(Y_Test, preds) 
    print(f'Testing accuracy:{skm.accuracy_score(Y_Test, preds):10.6f}\n')

    if wand_config == 1:
        wandb.log({"VFI-1": int(VFI[sub_test][0][0]),
                   "Validation Accuracy": learn.validate()[1],
                   "Testing Accuracy": skm.accuracy_score(Y_Test, preds)})
        run.finish()

In [ ]:
#
inp, train_probas, _, train_preds = learn.get_X_preds(X_Train.reshape((-1,1,48)), 
                                                      with_input=True, 
                                                      with_decoded=True)

In [ ]:
#
learn.predict(X_Train.reshape((-1,1,48)), with_input=True)

In [ ]:
X_Test.reshape((165,1,48)).shape

In [ ]:
inp, test_probas, _, test_preds = learn.get_X_preds(X_Test.reshape((165,1,48)), with_input=True, with_decoded=True)


In [ ]:
print(model)

In [ ]:
dls.dataloaders

In [ ]:
print(dsets.train[5001][1])

In [ ]:
dsets.train[0][0].numpy()

In [ ]:
inp, test_probas , _, test_preds  = learn.get_X_preds(X_Test.reshape((-1,1,48)), 
                                                      with_input=True, 
                                                      with_decoded=True)

In [ ]:
test_probas.numpy()